In [1]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph.message import add_messages
from pydantic import BaseModel
from typing import List, Literal, Annotated
from datetime import date
import json 

# tools.py 파일에서 TOOLS를 임포트
from tools import TOOLS

# =================== Embedding & VectorStore ===================
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")
vector_store = Chroma(
    embedding_function=embedding_model,
    collection_name="rag_chatbot",
    persist_directory="vector_store/chroma/rag_chatbot"
)
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

# =================== Prompt Chains ===================
hyde_chain = LLMChain(
    llm=ChatOpenAI(model_name="gpt-4.1", streaming=True),
    prompt=ChatPromptTemplate.from_template("""#Instruction:
다음 질문에 대해서 완전하고 상세한 답변으로 실제 사실에 기반해서 작성해주세요.
질문과 관련된 내용만으로 작성합니다.
답변과 직접적인 연관성이 없는 내용은 답변에 포함시키지 않습니다.

# 질문:
{query}
""")
)

llm_check_chain = LLMChain(
    llm=ChatOpenAI(model_name="gpt-4.1", streaming=True),
    prompt=ChatPromptTemplate.from_template("""
다음 Context는 사용자의 질문에 대해 충분한 정보를 제공하고 있는가?
질문: {query}
Context: {context}

위 Context만으로 정확하고 신뢰할 수 있는 답변이 가능한 경우 'Y',
불충분하다면 'N',
관련 없는 경우 'llm_message'로 답해주세요.

정확하게 하나의 문자만 출력하세요.
""")
)

# LLM이 도구 결과 요약 및 사용자 정보 기반 답변 생성을 위한 새로운 체인
response_synthesis_llm = ChatOpenAI(model_name="gpt-4.1", streaming=True)
response_synthesis_prompt = ChatPromptTemplate.from_messages([
    ("system", f"""
    당신은 사용자에게 친절하고 정확하게 답변하는 AI 어시스턴트입니다.
    오늘 날짜는 {date.today().strftime('%Y년 %m월 %d일')}입니다.
    사용자의 질문과, 제공된 정보 또는 대화 기록을 바탕으로 답변을 완성하세요.
    사용자 이름이 주어졌다면 답변에 친근하게 활용하세요.
    
    # 사용자 정보:
    이름: {{user_name}}
    
    # 대화 기록:
    {{chat_history}}
    
    # 제공된 정보 (도구 실행 결과 또는 RAG 컨텍스트):
    {{context}}
    
    # 답변:
    """),
    ("human", "{query}")
])
response_synthesis_chain = response_synthesis_prompt | response_synthesis_llm


# 엔티티 추출을 위한 LLM 체인
entity_extraction_llm = ChatOpenAI(model_name="gpt-4.1-mini")
entity_extraction_prompt = ChatPromptTemplate.from_template("""
주어진 대화에서 사용자의 이름을 정확히 추출해주세요. 이름만 추출하며, "이름:", "내 이름은"과 같은 접두사는 포함하지 마세요.
이름을 찾을 수 없다면 "None"이라고 답변하세요.

예시:
대화: 안녕 나 강감찬이야
이름: 강감찬

대화: 안녕하세요
이름: None

대화: 내 이름은 김철수야
이름: 김철수

대화: {dialogue}
이름:
""")
entity_extraction_chain = entity_extraction_prompt | entity_extraction_llm

# =================== GraphState ===================
class GraphState(BaseModel):
    messages: Annotated[List, add_messages]
    query: str = ""
    hyde_answer: str = ""
    context_docs: List = []
    context_str: str = ""
    tool_result: str = ""
    route: Literal["use_rag", "use_tool", "llm_message", "tool_call", "synthesize_response"] = "use_rag" 
    final_answer: str = ""
    source: str = ""
    user_name: str = "" 

# =================== Helper Functions ===================
def format_docs(docs: list) -> str:
    return "\n\n".join(doc.page_content for doc in docs)

def run_hyde(state: GraphState):
    latest_message = state.messages[-1].content
    
    hyde_result = hyde_chain.invoke({"query": latest_message})
    
    if isinstance(hyde_result, dict):
        hyde_answer = hyde_result.get("text", hyde_result.get("answer", str(hyde_result)))
    else:
        hyde_answer = str(hyde_result)
            
    print(f"DEBUG: Hyde Answer: '{hyde_answer}'")
    
    return {
        "query": latest_message, # 현재 턴의 질문을 query로 설정
        "hyde_answer": hyde_answer,
        "messages": state.messages + [AIMessage(content=hyde_answer)],
    }

def run_retriever(state: GraphState):
    docs = retriever.invoke(state.hyde_answer)
    
    if not docs:
        print("DEBUG: No documents found by retriever. Routing to tool or fallback.")
        return {"context_docs": [], "context_str": "", "route": "tool_call"} 
        
    context_str = format_docs(docs)
    print(f"DEBUG: Retrieved Context: '{context_str[:200]}...'")
    return {"context_docs": docs, "context_str": context_str, "route": "synthesize_response"}

def route_decision(state: GraphState):
    llm_check_result = llm_check_chain.invoke({"query": state.query, "context": state.context_str})
    
    decision_raw = ""
    if isinstance(llm_check_result, dict):
        decision_raw = llm_check_result.get("text", llm_check_result.get("answer", str(llm_check_result)))
    else:
        decision_raw = str(llm_check_result)
    
    decision = decision_raw.strip().lower()

    print(f"DEBUG: LLM Check Decision Raw: '{decision_raw}'")
    print(f"DEBUG: LLM Check Decision Processed: '{decision}'")

    if decision == "y":
        return {"route": "use_rag"}
    elif decision == "n":
        return {"route": "tool_call"}
    else:
        # LLM이 직접 답변해야 하는 경우에도 synthesize_response를 거치도록 변경
        # query와 messages를 바탕으로 답변을 생성하게 함
        return {"route": "synthesize_response"} 

# =================== Tool Node Functions ===================
tool_calling_llm = ChatOpenAI(model_name="gpt-4.1-mini", streaming=True).bind_tools(TOOLS)

def call_tool_llm(state: GraphState):
    print(f"DEBUG: Calling tool_calling_llm with query: {state.query}")
    response = tool_calling_llm.invoke(state.query)
    
    if response.tool_calls:
        print(f"DEBUG: Tool Call detected: {response.tool_calls}")
        return {"messages": state.messages + [response]}
    else:
        print(f"DEBUG: No tool call, LLM responded directly: {response.content}")
        # LLM이 직접 답변한 경우에도 synthesize_response로 넘겨서 최종 답변 포맷팅
        return {
            "tool_result": response.content, # LLM의 직접 응답을 tool_result에 저장하여 context로 활용
            "messages": state.messages + [response],
            "source": "llm_tool_fallback",
            "route": "synthesize_response" # synthesize_response로 라우팅
        }

def process_tool_result(state: GraphState):
    tool_message = None
    for msg in reversed(state.messages):
        if isinstance(msg, ToolMessage):
            tool_message = msg
            break

    if tool_message:
        tool_output_raw = tool_message.content
        tool_name = tool_message.name
        tool_id = tool_message.tool_call_id

        print(f"DEBUG: Tool execution result for tool '{tool_name}' (ID: {tool_id}): '{tool_output_raw[:200]}...'")
        
        processed_tool_output = ""
        source = tool_name 

        # search_web 도구의 출력 처리
        if tool_name == "search_web":
            try:
                parsed_output = json.loads(tool_output_raw)
                if isinstance(parsed_output, dict) and "results" in parsed_output:
                    search_results_list = parsed_output["results"]
                    if search_results_list:
                        formatted_results = []
                        for res in search_results_list:
                            content_snippet = res.get("content", "내용 없음")
                            if content_snippet:
                                content_snippet = content_snippet[:500] + "..." if len(content_snippet) > 500 else content_snippet
                            formatted_results.append(
                                f"**제목:** {res.get('title', '제목 없음')}\n"
                                f"**URL:** {res.get('url', 'URL 없음')}\n"
                                f"**내용:** {content_snippet}\n"
                                f"---"
                            )
                        processed_tool_output = "\n\n".join(formatted_results)
                    else:
                        processed_tool_output = "인터넷 검색 결과가 없습니다."
                else:
                    processed_tool_output = "인터넷 검색 결과 형식 오류."
            except json.JSONDecodeError as e:
                processed_tool_output = f"인터넷 검색 결과 JSON 파싱 오류: {e}"
                print(f"ERROR: Search web result JSON parsing failed: {e}")
            except Exception as e:
                processed_tool_output = f"인터넷 검색 결과 처리 중 오류 발생: {e}"
                print(f"ERROR: Search web result processing failed: {e}")
        else:
            processed_tool_output = tool_output_raw # LLM이 요약하도록 원본 전달

        if not processed_tool_output or \
           "접근할 수 없습니다" in processed_tool_output or \
           "검색결과가 없습니다" in processed_tool_output:
            
            return {
                "final_answer": "죄송합니다. 도구에서 관련 정보를 찾지 못했습니다.",
                "source": "llm",
                "tool_result": tool_output_raw 
            }
        
        return {
            "tool_result": processed_tool_output,
            "source": source,
            "route": "synthesize_response"
        }
    else:
        print("DEBUG: process_tool_result called but no recent ToolMessage found.")
        return {"final_answer": "도구 실행 결과를 처리할 수 없습니다.", "source": "llm"}


def synthesize_response(state: GraphState):
    print("DEBUG: Entering synthesize_response node.")
    context_to_synthesize = state.context_str 
    if state.tool_result: # tool_result가 있다면 (도구 호출 또는 LLM 직접 응답)
        context_to_synthesize = state.tool_result
        print(f"DEBUG: Synthesizing with Tool Result (or direct LLM response): {context_to_synthesize[:200]}...")
    elif not context_to_synthesize: # RAG 컨텍스트도 없고 tool_result도 없는 경우
        print("DEBUG: No context or tool result to synthesize with.")
        # 이 경우 LLM이 chat_history만으로 답변 시도
        context_to_synthesize = "" 

    try:
        # chat_history를 프롬프트에 포함하여 LLM이 대화 흐름을 이해하도록 함
        # LangChain의 messages 리스트를 그대로 전달
        final_answer_message = response_synthesis_chain.invoke({
            "query": state.query,
            "context": context_to_synthesize,
            "user_name": state.user_name,
            "chat_history": state.messages[:-1] # 마지막 사용자 메시지 제외 (현재 쿼리이므로)
        })
        final_answer_text = final_answer_message.content
        print(f"DEBUG: Synthesized Final Answer: '{final_answer_text}'")
        
        # LLM_tool_fallback이 synthesize_response로 넘어왔을 때 source를 llm으로 변경
        source_for_output = state.source
        if source_for_output == "llm_tool_fallback":
            source_for_output = "llm"

        return {
            "final_answer": final_answer_text,
            "source": source_for_output
        }
    except Exception as e:
        print(f"ERROR: Response synthesis failed: {e}")
        return {"final_answer": "죄송합니다. 답변을 생성하는 데 실패했습니다.", "source": "llm"}


def extract_user_name(state: GraphState):
    last_human_message_content = ""
    for msg in reversed(state.messages):
        if isinstance(msg, HumanMessage):
            last_human_message_content = msg.content
            break

    if last_human_message_content:
        name_extraction_result = entity_extraction_chain.invoke({"dialogue": last_human_message_content})
        extracted_name_raw = name_extraction_result.content.strip()
        
        # '이름:' 접두사 제거
        if extracted_name_raw.startswith("이름:"):
            extracted_name = extracted_name_raw[len("이름:"):].strip()
        else:
            extracted_name = extracted_name_raw
        
        print(f"DEBUG: Extracted name raw: '{extracted_name_raw}'")
        print(f"DEBUG: Extracted name processed: '{extracted_name}'")
        
        if extracted_name.lower() != "none" and extracted_name:
            print(f"DEBUG: User name extracted: '{extracted_name}'")
            return {"user_name": extracted_name}
    
    print("DEBUG: No user name extracted or 'None'.")
    return {"user_name": state.user_name}

def fallback_node(state: GraphState):
    final_answer = state.final_answer or "죄송합니다. 관련 정보를 찾지 못했습니다."
    print(f"DEBUG: Fallback Answer: '{final_answer}'")
    
    # fallback 시에도 synthesize_response를 거치도록 변경
    # fallback은 일반적으로 최종 답변을 내지 않고, LLM에게 최종 답변을 맡기는 역할
    return {"final_answer": final_answer, "source": "llm"} # 직접 답변하지 않고, synthesize_response로 넘어가야 함

# =================== Graph Setup ===================
checkpointer = MemorySaver()
graph = StateGraph(GraphState)

graph.add_node("extract_name", extract_user_name) 
graph.add_node("hyde", run_hyde)
graph.add_node("retrieve", run_retriever)
graph.add_node("check_route", route_decision)
graph.add_node("call_tool_llm", call_tool_llm)
graph.add_node("process_tool_result", process_tool_result)
from langgraph.prebuilt import ToolNode
tool_node = ToolNode(TOOLS)
graph.add_node("tool_runner", tool_node)
graph.add_node("synthesize_response", synthesize_response)
graph.add_node("fallback", fallback_node) # fallback 노드는 synthesize_response로 연결

graph.set_entry_point("extract_name")

graph.add_edge("extract_name", "hyde")

graph.add_edge("hyde", "check_route")

graph.add_conditional_edges("check_route", lambda state: state.route, {
    "use_rag": "retrieve",
    "tool_call": "call_tool_llm",
    "llm_message": "synthesize_response" # llm_message인 경우 synthesize_response로 바로 이동
})

graph.add_edge("retrieve", "synthesize_response")

graph.add_conditional_edges(
    "call_tool_llm",
    # LLM이 도구 호출을 하지 않고 직접 응답한 경우 바로 synthesize_response로 이동
    lambda state: "tool_runner" if hasattr(state.messages[-1], 'tool_calls') and state.messages[-1].tool_calls else "synthesize_response",
    {
        "tool_runner": "tool_runner",
        "synthesize_response": "synthesize_response" # LLM이 직접 응답한 경우
    }
)

graph.add_edge("tool_runner", "process_tool_result")
graph.add_edge("process_tool_result", "synthesize_response")

graph.add_edge("synthesize_response", END)
graph.add_edge("fallback", END) # fallback은 이제 직접 END로 갈 수 있음. (synthesize_response로 라우팅하는 대신)

final_graph = graph.compile(checkpointer=checkpointer)

C:\Users\Playdata\AppData\Local\Temp\ipykernel_26016\1204594300.py:27: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  hyde_chain = LLMChain(


In [21]:
import os
import random
import pandas as pd
import numpy as np  # numpy 임포트
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_community.document_loaders import DirectoryLoader, TextLoader, PyPDFLoader, CSVLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pydantic import BaseModel, Field

from ragas import evaluate
from ragas.metrics import (
    Faithfulness,
    AnswerRelevancy,
    ContextRecall,
    ContextPrecision,
)
from ragas.llms import LangchainLLMWrapper
from datasets import Dataset

# --- 1. 환경 설정 ---
print("[1/7] 환경 설정을 시작합니다...")
load_dotenv()

EVAL_LLM = ChatOpenAI(model="gpt-4.1-mini", temperature=0)
EVAL_EMBEDDING_MODEL = OpenAIEmbeddings(model="text-embedding-3-large")

KNOWLEDGE_BASE_FOLDER_PATH = "data/"
NUM_EVAL_SAMPLES = 10
CHROMA_PERSIST_DIRECTORY = "vector_store/chroma/copyright_law"
CHROMA_COLLECTION_NAME = "copyright_law_rag"

# --- 2. 문서 로드 및 RAG 체인 구성 ---
print(f"[2/7] '{KNOWLEDGE_BASE_FOLDER_PATH}' 폴더에서 모든 문서를 로드하여 RAG 시스템을 구성합니다...")

txt_loader = DirectoryLoader(KNOWLEDGE_BASE_FOLDER_PATH, glob="**/*.txt", loader_cls=TextLoader, show_progress=True)
txt_docs = txt_loader.load()
print(f"Found {len(txt_docs)} .txt files.")

pdf_loader = DirectoryLoader(KNOWLEDGE_BASE_FOLDER_PATH, glob="**/*.pdf", loader_cls=PyPDFLoader, show_progress=True)
pdf_docs = pdf_loader.load()
print(f"Found {len(pdf_docs)} .pdf files.")

csv_loader = DirectoryLoader(KNOWLEDGE_BASE_FOLDER_PATH, glob="**/*.csv", loader_cls=CSVLoader, loader_kwargs={'encoding': 'utf-8', 'source_column': 'contents'}, show_progress=True)
csv_docs = csv_loader.load()
print(f"Found {len(csv_docs)} .csv files.")

json_loader = DirectoryLoader(KNOWLEDGE_BASE_FOLDER_PATH, glob="**/*.json", loader_cls=JSONLoader, loader_kwargs={'jq_schema': '.[] | .content', 'text_content': False}, show_progress=True)
json_docs = json_loader.load()
print(f"Found {len(json_docs)} documents from .json files.")

all_docs = txt_docs + pdf_docs + csv_docs + json_docs
print(f"Total documents loaded: {len(all_docs)}")

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
split_docs = splitter.split_documents(all_docs)
print(f"Total chunks created: {len(split_docs)}")

print("Initializing Chroma DB...")
vector_store = Chroma(embedding_function=EVAL_EMBEDDING_MODEL, collection_name=CHROMA_COLLECTION_NAME, persist_directory=CHROMA_PERSIST_DIRECTORY)

BATCH_SIZE = 128
num_batches = (len(split_docs) - 1) // BATCH_SIZE + 1
for i in range(num_batches):
    start_index = i * BATCH_SIZE
    end_index = start_index + BATCH_SIZE
    batch_docs = split_docs[start_index:end_index]
    print(f"Adding batch {i+1}/{num_batches} to Chroma DB...")
    vector_store.add_documents(batch_docs)

print("All documents have been added to Chroma DB.")
retriever = vector_store.as_retriever()

RAG_PROMPT_TEMPLATE = """
#Instruction:
당신은 전문가입니다. 주어진 Context 내용을 바탕으로 질문에 대해 정확하고 근거에 기반하여 답변하세요.
Context에 질문과 관련된 내용이 없으면 "관련 정보를 찾을 수 없습니다."라고 답변하세요.

#Context:
{context}

#질문:
{query}
"""
rag_prompt = PromptTemplate.from_template(RAG_PROMPT_TEMPLATE)

rag_chain_for_eval = (
    {"context": retriever, "query": RunnablePassthrough()}
    | RunnableLambda(lambda x: {
        "context": "\n\n".join([doc.page_content for doc in x["context"]]),
        "query": x["query"],
        "contexts": [doc.page_content for doc in x["context"]]
    })
    | {
        "answer": rag_prompt | EVAL_LLM | StrOutputParser(),
        "contexts": lambda x: x["contexts"]
    }
)

# --- 3. 평가 데이터셋 자동 생성 ---
print("[3/7] 평가 데이터셋을 자동으로 생성합니다...")

class QASchema(BaseModel):
    question: str = Field(description="주어진 문맥을 바탕으로 생성된 사용자 질문")
    ground_truth: str = Field(description="생성된 질문에 대한, 문맥에 충실한 정답")

QA_GEN_PROMPT = """
#Instruction:
당신은 '크리에이터를 위한 저작권법 RAG 챗봇'의 성능을 평가할 데이터셋을 만드는 AI입니다.
당신의 임무는 유튜버, 블로거, 웹툰 작가, 디자이너 등 콘텐츠 크리에이터의 관점에서 질문과 정답 쌍을 생성하는 것입니다.
생성할 질문은 크리에이터들이 자신의 창작 활동(예: 영상 제작, 굿즈 판매, 폰트 사용, 2차 창작) 중에 마주칠 수 있는
구체적이고 현실적인 저작권법 관련 궁금증이어야 합니다.
- 질문과 정답은 반드시 주어진 [Context]에 있는 정보만을 근거로 생성해야 합니다.
- 정답은 [Context]를 바탕으로 질문에 대해 완전한 문장으로 작성해주세요.
- "문서에 따르면", "제공된 내용에 의하면" 과 같은 표현은 질문에 포함하지 마세요.

# 좋은 질문 예시:
- "제가 만든 유튜브 영상에 다른 사람의 음악 5초 정도를 배경음악으로 써도 저작권에 걸리나요?"
- "인터넷에서 본 폰트를 제 상업용 굿즈에 사용해도 되나요?"
- "다른 사람의 글을 리뷰하면서 일부 인용하는 건 어디까지 허용되나요?"

#Context:
{context}

#Output Indicator:
{format_instructions}
"""
parser = JsonOutputParser(pydantic_object=QASchema)
qa_gen_prompt = PromptTemplate(
    template=QA_GEN_PROMPT,
    input_variables=["context"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
qa_gen_chain = qa_gen_prompt | EVAL_LLM | parser

random.shuffle(split_docs)
selected_docs_for_qa = split_docs[:min(NUM_EVAL_SAMPLES, len(split_docs))]
eval_data_list = qa_gen_chain.batch([{"context": doc.page_content} for doc in selected_docs_for_qa])
print(f"Generated {len(eval_data_list)} Q&A pairs for evaluation.")


[1/7] 환경 설정을 시작합니다...
[2/7] 'data/' 폴더에서 모든 문서를 로드하여 RAG 시스템을 구성합니다...


0it [00:00, ?it/s]


Found 0 .txt files.


 80%|████████  | 12/15 [01:07<00:37, 12.54s/it]Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 50 0 (offset 0)
Ignoring wrong pointing object 68 0 (offset 0)
100%|██████████| 15/15 [01:09<00:00,  4.61s/it]


Found 1400 .pdf files.


...

Found 0 .csv files.


100%|██████████| 2/2 [00:00<00:00, 105.14it/s]


Found 31 documents from .json files.
Total documents loaded: 1431
Total chunks created: 2930
Initializing Chroma DB...
Adding batch 1/23 to Chroma DB...
Adding batch 2/23 to Chroma DB...
Adding batch 3/23 to Chroma DB...
Adding batch 4/23 to Chroma DB...
Adding batch 5/23 to Chroma DB...
Adding batch 6/23 to Chroma DB...
Adding batch 7/23 to Chroma DB...
Adding batch 8/23 to Chroma DB...
Adding batch 9/23 to Chroma DB...
Adding batch 10/23 to Chroma DB...
Adding batch 11/23 to Chroma DB...
Adding batch 12/23 to Chroma DB...
Adding batch 13/23 to Chroma DB...
Adding batch 14/23 to Chroma DB...
Adding batch 15/23 to Chroma DB...
Adding batch 16/23 to Chroma DB...
Adding batch 17/23 to Chroma DB...
Adding batch 18/23 to Chroma DB...
Adding batch 19/23 to Chroma DB...
Adding batch 20/23 to Chroma DB...
Adding batch 21/23 to Chroma DB...
Adding batch 22/23 to Chroma DB...
Adding batch 23/23 to Chroma DB...
All documents have been added to Chroma DB.
[3/7] 평가 데이터셋을 자동으로 생성합니다...
Generated 10

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]


    RAG 성능 평가 요약 (DataFrame)
                      Score
Metric                     
faithfulness       0.866818
answer_relevancy   0.637050
context_recall     0.800000
context_precision  0.800000

     상세 평가 결과 (DataFrame)
                                          user_input  \
0                  2024년 12월에 새로 제정된 저작권 관련 법률이 있나요?   
1  내가 만든 유튜브 영상이 다른 사람에게 무단으로 도용되었을 때 어떻게 대응할 수 있나요?   
2  AI가 합의 없이 젊은 여성의 얼굴을 이용해 포르노 영상을 생성하는 것은 저작권법이...   
3  2005년 제정된 Artists’ Rights and Theft Prevention...   
4        제가 만든 교육용 영상에 교과서의 삽화 일부를 사용해도 저작권 문제가 없나요?   
5       국가나 지방자치단체가 작성한 저작물을 유튜브 영상에 자유롭게 사용할 수 있나요?   
6  제가 만든 유튜브 영상에 다른 사람의 음악을 배경음악으로 사용하려면 어떤 권리를 확...   
7  제가 오래전에 만든 작품인데, 저작권 보호 기간이 아직 끝나지 않았다면 이 법이 적...   
8      웹소설을 원작으로 한 드라마가 저작권 침해가 되려면 어떤 부분이 유사해야 하나요?   
9   해외에서 정식으로 구매한 저작권 보호 상품을 국내에서 재판매해도 저작권 침해가 되나요?   

                                  retrieved_contexts  \
0  [저작권의 \n내용\n2024\n저작권\n상담 사례집\nKOREA COPYRIGHT...   
1  [144   1인 미디어 창작자를 위한 저작권 안내서 \n내 창작물을 무단 도

In [ ]:
# --- 4. RAG 시스템 응답 생성 ---
print("[4/7] 생성된 질문으로 RAG 시스템의 응답을 생성합니다...")
questions = [item['question'] for item in eval_data_list]
rag_responses = rag_chain_for_eval.batch(questions)

# --- 5. RAGAs 평가 데이터셋 구성 ---
print("[5/7] RAGAs 평가를 위한 최종 데이터셋을 구성합니다...")
data_for_ragas = {
    "question": questions,
    "answer": [resp['answer'] for resp in rag_responses],
    "contexts": [resp['contexts'] for resp in rag_responses],
    "ground_truth": [item['ground_truth'] for item in eval_data_list]
}
ragas_dataset = Dataset.from_dict(data_for_ragas)

# --- 6. RAGAs로 성능 평가 실행 ---
print("[6/7] RAGAs 프레임워크로 시스템 성능 평가를 시작합니다...")

ragas_llm = LangchainLLMWrapper(EVAL_LLM)

metrics_to_use = [
    Faithfulness(llm=ragas_llm),
    AnswerRelevancy(llm=ragas_llm),
    ContextRecall(llm=ragas_llm),
    ContextPrecision(llm=ragas_llm),
]
result = evaluate(
    dataset=ragas_dataset,
    metrics=metrics_to_use,
    llm=ragas_llm,
    embeddings=EVAL_EMBEDDING_MODEL
)

# --- 7. 최종 결과 출력 (수정된 부분) ---

# 각 평가지표는 점수 리스트를 반환하므로, 평균(mean)을 계산하여 요약 점수를 만듭니다.
summary_scores = {
    'faithfulness': np.mean(result['faithfulness']),
    'answer_relevancy': np.mean(result['answer_relevancy']),
    'context_recall': np.mean(result['context_recall']),
    'context_precision': np.mean(result['context_precision']),
}
df_summary = pd.DataFrame.from_dict(summary_scores, orient='index', columns=['Score'])
df_summary.index.name = 'Metric'

# 상세 결과는 to_pandas()를 사용하여 각 샘플별 점수를 확인합니다.
df_detailed_result = result.to_pandas()

# --- 최종 결과 출력 ---
print("\n" + "="*35)
print("    RAG 성능 평가 요약 (DataFrame)")
print("="*35)
print(df_summary)

print("\n" + "="*40)
print("     상세 평가 결과 (DataFrame)")
print("="*40)
print(df_detailed_result)